In [1]:
import numpy as np
import jax
import jax.numpy as jnp
# import scipy.stats as ss
import tensorly as tl
from tensorly.regression.tucker_regression import TuckerRegressor

# import matplotlib.pyplot as plt
# %matplotlib inline
# from IPython.display import display
# import sympy; sympy.init_printing()

# def display_matrix(m):
#     display(sympy.Matrix(m))

In [2]:
try:
    tl.set_backend('jax')
except:
    pass
tl.set_backend('jax')

In [3]:
SEED = 123

In [4]:
sample = np.genfromtxt('references/sample.csv', delimiter=',')

In [5]:
S = jnp.transpose(sample)

# check distribution
print(S.shape) # paths, time

(10000, 100)


In [6]:
print(jnp.mean(S[:,-1]))
print(jnp.std(S[:,-1]))

# 1.05
# 0.2667

1.0518324048923096
0.26522389889480086


In [7]:
N = S.shape[1]          # number of time steps
r = 0.05       # interest rate
K = 1          # strike 
T = 1          # Maturity

dt = T/(N-1)          # time interval
df = jnp.exp(-r * dt)  # discount factor per time interval

In [8]:
# # jax assignment
# x.at[i].set(y)

In [9]:
# # Previous sample
# H = np.maximum(K - S, 0)           # intrinsic values for put option

# V = np.zeros_like(H)               # value matrix
# V[:,-1] = H[:,-1]

# # Valuation by Longstaff-Schwartz (LS) Method
# for t in range(N-2, 0, -1):

#     good_paths = H[:,t] > 0        # paths where the intrinsic value is positive 
#                                    # the regression is performed only on these paths 
    
#     rg = jnp.polyfit( S[good_paths, t], V[good_paths, t+1] * df, 2)    # polynomial regression
#     C = jnp.polyval( rg, S[good_paths,t] )                             # evaluation of regression 
    
#     exercise = np.zeros( len(good_paths), dtype=bool)    # initialize
#     exercise[good_paths] = H[good_paths,t] > C           # paths where it is optimal to exercise
    
#     V[exercise,t] = H[exercise,t]                        # set V equal to H where it is optimal to exercise 
#     V[exercise,t+1:] = 0                                 # set future cash flows, for that path, equal to zero  
#     discount_path = (V[:,t] == 0)                        # paths where we didn't exercise 
#     V[discount_path,t] = V[discount_path,t+1] * df       # set V[t] in continuation region
    
# V0 = jnp.mean(V[:,1]) * df  # discounted expectation of V[t=1]
# print("Example price= ", V0)

# # 0.07364798697497656

In [10]:
H = jnp.maximum(K - S, 0) # intrinsic values for put option
V = jnp.zeros_like(H) # value matrix
V = V.at[:,-1].set(H[:,-1]) # when option expires, its value equals intrinsic value

# Reverse computing
for t in range(N-2, 0, -1):
    good_paths = H[:,t] > 0 # paths where intrinsic values are positive 
                            # (removing paths where intrinsic values are 0)
                            # regression is performed only on these paths
    
    # Valuation by Tucker Regression
    X = S[good_paths, t].reshape((-1,1,1)) # reshape to fill up dimensions
    y = V[good_paths, t+1] * df
    estimator = TuckerRegressor(weight_ranks=[1]*X.ndim, verbose=False, random_state=SEED) # hyperparameter: reg_W (default: 1) 
    estimator.fit(X, y)
    fitted_values = estimator.predict(X) # predicted value of option (discounted, as `y` is discounted)
    
    exercise = jnp.zeros(len(good_paths)).astype(bool) # initialize
    # early exercise when value of early exericse at t > predicted value at t+1 (discounted to t)
    exerise = exercise.at[good_paths].set(H[good_paths,t] > fitted_values)

    V = V.at[exercise,t].set(H[exercise,t]) # set V equal to H where it is optimal to exercise 
    V = V.at[exercise,t+1:].set(0) # set future cash flows, for that path, equal to zero  
    discount_path = V[:,t] == 0 # paths where we didn't exercise
    V = V.at[discount_path,t].set(V[discount_path,t+1] * df) # set V[t] in continuation region
    
V0 = jnp.mean(V[:,1]) * df
print(f"option price = {V0}")

option price = 0.07356839794765795


In [11]:
print(H.round(2))

[[0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.18 0.15 0.12]
 [0.   0.   0.   ... 0.15 0.16 0.2 ]
 ...
 [0.   0.   0.   ... 0.23 0.26 0.27]
 [0.   0.   0.   ... 0.   0.   0.  ]
 [0.   0.   0.   ... 0.   0.   0.  ]]


In [12]:
# 0.0797